In [3]:
import numpy as np
import cv2
import random
import imageio
import os
import torch

In [4]:
def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)
# plot_one_box(bboxes, overlayImage, label=label, color=color, line_thickness=line_thickness, bottom_label=bottom_label)
def plot_one_box(x, img, color=None, label=None, line_thickness=None, bottom_label=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1# line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    
    tf = max(tl - 1, 1)# font thickness
    if label:
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c4 = c1[0] + t_size[0], c1[1] - t_size[1] - 3# filled
        cv2.rectangle(img, c1, c4, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    if bottom_label:
        a_size = cv2.getTextSize(bottom_label, 0, fontScale=tl / 4, thickness=tf)[0]
        c3 = c1[0] + a_size[0], c2[1] + a_size[1] + 3
        cv2.rectangle(img, (c1[0], c2[1]), c3, color, -1, cv2.LINE_AA)
        cv2.putText(img, bottom_label, (c1[0], c2[1] + 12), 0, tl / 4, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def plot_text(up_text,img,c1=(546,40),c2=(550,50),color=(0,0,255)):
    tl = 3
    tf = max(tl - 1, 1)# font thickness
    up_text_size = cv2.getTextSize(up_text, 0, fontScale=tl / 3, thickness=tf)[0]
    c4 = c1[0] + up_text_size[0], c1[1] - up_text_size[1] - 3# filled
    cv2.rectangle(img, c1, c4, color, -1, cv2.LINE_AA)
    cv2.putText(img, up_text, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

In [5]:
def resize_img(im, target_width = 640):
    h,w,_  = im.shape
    target_height = int(h / w * target_width)
    im = cv2.resize(im , (target_width , target_height), interpolation = cv2.INTER_AREA)  
    return im,target_height,target_width

In [6]:
def xyxy_xywh(x1,y1,x2,y2):
    w = abs(x2-x1)
    h = abs(y2-y1)
    x,y = x1+w/2,y1+h/2
    return x,y,w,h
def xywh_xyxy(x,y,w,h):
    x1 = x-w/2
    y1 = y-h/2
    x2 = x+w/2
    y2 = y+h/2
    return int(x1),int(y1),int(x2),int(y2)

def scale_con(x,y,w,h,old,new):
    # old : tuple(height,width)
    x = int(x* old[0]/new[0])
    y = int(y*old[1]/new[1])
    h = int(h*old[0]/new[0])
    w = int(w*old[1]/new[1])
    return x,y,w,h 
def isInPolygon(PointF, polygon):
    i = 0
    c = False
    j = len(polygon) - 1
    for i in range(len(polygon)):        
        if (((polygon[i][1] > PointF[1]) != (polygon[j][1] > PointF[1])) and 
        (PointF[0] < (polygon[j][0] - polygon[i][0]) * (PointF[1] - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0])):
            c = not(c)
        j = i
    return c

In [23]:
model = torch.hub.load("./yolov5", 'custom', path='./yolov5/runs/train/exp3/weights/best.pt', source='local')
model.conf = 0.5

# model = torch.hub.load("../Human_Detection/yolov5", "yolo5m",source="local")  # or yolov5n - yolov5x6, custom
# '.', 'custom', path='/path/to/yolov5/runs/train/exp5/weights/best.pt', source='local'

YOLOv5 🚀 2023-12-25 Python-3.8.3 torch-1.12.0+cu102 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [13]:
from boxmot import tracker_zoo

In [14]:
from pathlib import Path

In [15]:
# Tracker Initialization
# Tracker options: 'deepocsort, botsort, strongsort, ocsort, bytetrack')
path = Path("./osnet_x1_0_dukemtmcreid.pt").absolute()
config = tracker_zoo.get_tracker_config('botsort')
tracker = tracker_zoo.create_tracker('botsort', "./boxmot/configs/botsort.yaml", path,
                                     "cpu", False, False)

2023-12-28 07:22:40.507 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "/media/umair/New Volume1/Object_Detection_tracking/Human_Detection_Tracking_Counting/osnet_x1_0_dukemtmcreid.pt"


In [16]:
cap = cv2.VideoCapture("./video/student_lobby.mp4")
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('./video/student_lobby_counted.mp4', fourcc, fps, (frame_width, frame_height))

# Polygon Area for counting tracking


area1 = np.array([[351,380],[302,713],[465,711],[487,395]])
area2 = np.array([[494,395],[472,711],[608,711],[630,402]])

# Polygon Detection Area
det_roi = np.array([[341,375],[292,710],[873,710],[835,375]])

left_count = []
right_count = []
area1_tracks = []
area2_tracks = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        
        plot_text("Left Count: {}".format(len(left_count)),frame,c1=(500,50),c2=(510,60),color=(0,0,255))
        plot_text("Right Count: {}".format(len(right_count)),frame,c1=(500,85),c2=(510,95),color=(0,0,255))
        cv2.polylines(frame, [area1], True, (0,0,255), 2)
        cv2.polylines(frame, [area2], True, (0,0,255), 2)
        cv2.polylines(frame, [det_roi], True, (0,255,0), 2)
        
        img,res_height,res_width = resize_img(frame)
        results = model(img)
        preds = np.array(results.xyxy[0])
        bboxes = []
        
        for ind,(x1,y1,x2,y2,conf,cls) in enumerate(preds):
#             print(ind,x1,y1,x2,y2,conf,cls)
            x1,y1,x2,y2 = scale_con(x1,y1,x2,y2,(frame_height,frame_width),(res_height,res_width))
            if isInPolygon((x2,y2),det_roi):
                bboxes.append([x1,y1,x2,y2,conf,cls])
        
#         print(bboxes)
        if len(bboxes) == 0:
            bboxes = [[0,0,0,0,0.0,0]]
        outputs = tracker.update(np.array(bboxes),frame)
#         print(outputs)
        for x1,y1,x2,y2,tid,conf,_,_ in  outputs:
            plot_one_box(x=(x1,y1,x2,y2),img=frame,color=compute_color_for_id(int(tid))
                         ,label=str(int(tid)),line_thickness=1)
            
            if isInPolygon((x2,y2),area1):
                if tid not in area1_tracks:
                    area1_tracks.append(tid)
                    if tid in area2_tracks:
                        left_count.append(tid)
            elif isInPolygon((x2,y2),area2):
                if tid not in area2_tracks:
                    area2_tracks.append(tid)
                    if tid in area1_tracks:
                        right_count.append(tid)
                    
            
            
        output.write(frame)    
#         cv2.imshow('Frame',frame)
#         k =cv2.waitKey(0) & 0xFF
#         #press y to exit
#         if k==121:
#             break
#         else:
#             pass
    else:
        break
cap.release() 
# cv2.destroyAllWindows()
output.release()

In [17]:
# cap = cv2.VideoCapture("./video/pedestrian.webm")
# frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = cap.get(cv2.CAP_PROP_FPS)

# # Video writer

# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# output = cv2.VideoWriter('./video/crowd_street.mp4', fourcc, fps, (frame_width, frame_height))

# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == True:
#         output.write(frame) 
#     else: 
#         break
# cap.release()
# output.release()


In [18]:

# writing processed and unprocessed video file side by side
cap1 = cv2.VideoCapture("./video/student_lobby.mp4")
cap2 = cv2.VideoCapture("./video/student_lobby_counted.mp4")
frame_width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
target_width = frame_width//2
target_height = frame_height//2
fps = cap1.get(cv2.CAP_PROP_FPS)

# Video writer

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('./video/joined_video.mp4', fourcc, fps, (frame_width, target_height))


# target_frames = 800
print(frame_width,frame_height,fps)
print(target_width,target_height,fps)
while(cap1.isOpened()):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    if ret1 == True and ret2 == True :
        res_frame1 = cv2.resize(frame1 , (target_width , target_height), interpolation = cv2.INTER_AREA)  
        res_frame2 = cv2.resize(frame2 , (target_width , target_height), interpolation = cv2.INTER_AREA) 
        concat_frame = np.concatenate((res_frame1, res_frame2), axis=1)
        
        output.write(concat_frame)
#         cv2.imshow('Frame',concat_frame)
#         k =cv2.waitKey(0) & 0xFF
#         #press y to exit
#         if k==121:
#             break
#         else:
#             pass
    else:
        break
cap1.release()
cap2.release()
cv2.destroyAllWindows()
output.release()

1280 714 12.0
640 357 12.0


In [19]:
import imageio
import moviepy.editor as mp
from moviepy.editor import VideoFileClip

In [20]:
def convert_video_to_gif(input_video_path, output_gif_path, fps=25):
    video_clip = mp.VideoFileClip(input_video_path)
    video_clip.write_gif(output_gif_path, fps=fps)

In [21]:
video_file = "./video/joined_video.mp4"
output_gif_file = "./video/joined_video.gif"
convert_video_to_gif(video_file, output_gif_file)

MoviePy - Building file ./video/joined_video.gif with imageio.


In [27]:
def convert_half_video_to_gif(input_video_path, output_gif_path):
    # Load the video clip
    video_clip = VideoFileClip(input_video_path)

    # Get the duration of the video
    total_duration = video_clip.duration

    # Set the end time for the first half
    end_time = total_duration / 2
#     print(end_time,type(end_time))

    # Trim the video to the first half
    trimmed_clip = video_clip.subclip(0, 5.0)

    # Write the trimmed video as a GIF
    trimmed_clip.write_gif(output_gif_path)

video_file = "./video/output_crowd_street.mp4"
output_gif_file = "./video/clipped_crowd_street.gif"
convert_half_video_to_gif(video_file, output_gif_file)